# 1) Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 2) Read Dataset

In [2]:
books = pd.read_csv("dataset - amazon - books.csv",sep=";",error_bad_lines=False, encoding="latin-1")
ratings = pd.read_csv("dataset - amazon - ratings.csv",sep=";",error_bad_lines=False, encoding="latin-1")
users = pd.read_csv("dataset - amazon - users.csv",sep=";",error_bad_lines=False, encoding="latin-1")
books1 = books.drop(["Image-URL-S","Image-URL-M"],axis=1)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\I

In [3]:
print(books1.head())
print(books1.info())
print("---------------------------------------------------------------------------------------")

print(ratings.head())
print(ratings.info())
print("---------------------------------------------------------------------------------------")

print(users.head())
print(users.info())
print("---------------------------------------------------------------------------------------")

         ISBN                                         Book-Title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

            Book-Author Year-Of-Publication                   Publisher  \
0    Mark P. O. Morford                2002     Oxford University Press   
1  Richard Bruce Wright                2001       HarperFlamingo Canada   
2          Carlo D'Este                1991             HarperPerennial   
3      Gina Bari Kolata                1999        Farrar Straus Giroux   
4       E. J. W. Barber                1999  W. W. Norton &amp; Company   

                                         Image-URL-L  
0  http://images.amazon.com/images/P/0195153448.0...  
1  http://images.amazon.com/im

In [4]:
books1

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...


In [5]:
country = users["Location"].apply(lambda x:x.split(",")[-1]).value_counts()[:7].index.tolist()

In [6]:
country 

[' usa',
 ' canada',
 ' united kingdom',
 ' germany',
 ' spain',
 ' australia',
 ' italy']

In [7]:
users["country"] = users["Location"].apply(lambda x:x.split(",")[-1])

In [8]:
users1 = users[users["country"].isin(country)]

In [9]:
users1.head(10)

,User-ID,Location,Age,country
0,1,"nyc, new york, usa",NaN,usa
1,2,"stockton, california, usa",18.0,usa
4,5,"farnborough, hants, united kingdom",NaN,united kingdom
5,6,"santa monica, california, usa",61.0,usa
6,7,"washington, dc, usa",NaN,usa
7,8,"timmins, ontario, canada",NaN,canada
8,9,"germantown, tennessee, usa",NaN,usa
9,10,"albacete, wisconsin, spain",26.0,spain
10,11,"melbourne, victoria, australia",14.0,australia
11,12,"fort bragg, california, usa",NaN,usa


In [10]:
users1.shape

(233147, 4)

# Users with less than 50 ratings and books less than 100 ratings are excluded.

In [11]:
counts1 = ratings['User-ID'].value_counts()
ratings1 = ratings[ratings['User-ID'].isin(counts1[counts1 >= 50].index)]
counts2 = ratings['ISBN'].value_counts()
ratings2 = ratings[ratings['ISBN'].isin(counts2[counts2 >= 100].index)]

print(counts1)
print(ratings1)
print(counts2)
print(ratings2)



11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
271728        1
245123        1
234886        1
259466        1
187812        1
Name: User-ID, Length: 105283, dtype: int64
         User-ID        ISBN  Book-Rating
173       276847  0446364193            0
174       276847  3257200552            5
175       276847  3379015180            0
176       276847  3404145909            8
177       276847  3404148576            8
...          ...         ...          ...
1149744   276688  0836236688           10
1149745   276688  0892966548           10
1149746   276688  1551669315            6
1149747   276688  1575660792            7
1149748   276688  1580600506            0

[768472 rows x 3 columns]
0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
              ... 
0942668456       1
0553253964       1
0590246607       1
0725512377       1
0312155212       1
Name: ISBN, Length: 340556, dtype: int64

# Collaborative Filtering Using k-Nearest Neighbors(KNN)

In [138]:
combine_book_rating = pd.merge(ratings2, books1, on='ISBN')
combine_books = combine_book_rating.drop(["Book-Author","Year-Of-Publication","Publisher","Image-URL-L"],axis = 1)

In [139]:
combine_books

,User-ID,ISBN,Book-Rating,Book-Title
0,276727,0446520802,0,The Notebook
1,278418,0446520802,0,The Notebook
2,638,0446520802,0,The Notebook
3,3363,0446520802,0,The Notebook
4,7158,0446520802,10,The Notebook
...,...,...,...,...
137418,263877,0312971125,0,Crazy for You
137419,265115,0312971125,10,Crazy for You
137420,266056,0312971125,0,Crazy for You
137421,267249,0312971125,0,Crazy for You


In [140]:
book_ratings_count = (combine_books.groupby(by = ['Book-Title'])
                     
                     ['Book-Rating'].count().reset_index().rename(columns = {'Book-Rating' :'Total_Rating_Count'})[['Book-Title','Total_Rating_Count']])

book_ratings_count.sort_values(by = 'Total_Rating_Count',ascending=False).head(20)

,Book-Title,Total_Rating_Count
665,Wild Animus,2502
539,The Lovely Bones: A Novel,1295
476,The Da Vinci Code,883
18,A Painted House,831
544,The Nanny Diaries: A Novel,828
147,Divine Secrets of the Ya-Ya Sisterhood: A Novel,732
571,The Red Tent (Bestselling Backlist),723
92,Bridget Jones's Diary,713
596,The Summons,653
283,Life of Pi,632


In [141]:
combine_books1 = combine_books.merge(book_ratings_count,on = 'Book-Title')
combine_books1.head(10)

,User-ID,ISBN,Book-Rating,Book-Title,Total_Rating_Count
0,276727,0446520802,0,The Notebook,581
1,278418,0446520802,0,The Notebook,581
2,638,0446520802,0,The Notebook,581
3,3363,0446520802,0,The Notebook,581
4,7158,0446520802,10,The Notebook,581
5,8253,0446520802,10,The Notebook,581
6,9939,0446520802,0,The Notebook,581
7,11676,0446520802,10,The Notebook,581
8,12589,0446520802,9,The Notebook,581
9,13279,0446520802,10,The Notebook,581


In [202]:
popularity_threshold = 30
combine_books2 = combine_books1.query('Total_Rating_Count >= @popularity_threshold')
combine_books2.head(-1)

,User-ID,ISBN,Book-Rating,Book-Title,Total_Rating_Count
4,7158.0,0446520802,10.0,The Notebook,581.0
5,8253.0,0446520802,10.0,The Notebook,581.0
7,11676.0,0446520802,10.0,The Notebook,581.0
8,12589.0,0446520802,9.0,The Notebook,581.0
9,13279.0,0446520802,10.0,The Notebook,581.0
...,...,...,...,...,...
137399,227349.0,0312971125,9.0,Crazy for You,111.0
137404,236164.0,0312971125,7.0,Crazy for You,111.0
137405,236662.0,0312971125,8.0,Crazy for You,111.0
137407,242083.0,0312971125,6.0,Crazy for You,111.0


In [203]:
combine_books3 = combine_books2.merge(users1,on='User-ID')
combine_books3.shape

(52405, 8)

In [204]:
combine_books3 = combine_books2.merge(users,on='User-ID')
combine_books3.head()

,User-ID,ISBN,Book-Rating,Book-Title,Total_Rating_Count,Location,Age,country
0,7158.0,0446520802,10.0,The Notebook,581.0,"omaha, nebraska, usa",30.0,usa
1,7158.0,0446605239,10.0,The Notebook,581.0,"omaha, nebraska, usa",30.0,usa
2,7158.0,0446605484,8.0,Roses Are Red (Alex Cross Novels),230.0,"omaha, nebraska, usa",30.0,usa
3,7158.0,0316693235,5.0,Violets Are Blue,370.0,"omaha, nebraska, usa",30.0,usa
4,7158.0,074343627X,1.0,Dreamcatcher,452.0,"omaha, nebraska, usa",30.0,usa


In [205]:
countries_combine = combine_books3[combine_books3['Location'].str.contains('usa|canada|united kingdom|germany|spain|australia|italy')]
countries_combine = countries_combine.drop(['Age','Location'],axis = 1)
countries_combine.head()

,User-ID,ISBN,Book-Rating,Book-Title,Total_Rating_Count,country
0,7158.0,0446520802,10.0,The Notebook,581.0,usa
1,7158.0,0446605239,10.0,The Notebook,581.0,usa
2,7158.0,0446605484,8.0,Roses Are Red (Alex Cross Novels),230.0,usa
3,7158.0,0316693235,5.0,Violets Are Blue,370.0,usa
4,7158.0,074343627X,1.0,Dreamcatcher,452.0,usa


In [206]:
from scipy.sparse import csr_matrix
countries_combine = countries_combine.drop_duplicates(['User-ID', 'Book-Title'])
countries_combine_pivot = countries_combine.pivot(index= 'Book-Title',
                                                        columns = 'User-ID' ,
                                                        values = 'Book-Rating').fillna(0)
countries_combine_matrix = csr_matrix(countries_combine_pivot.values)
countries_combine_pivot.head()

User-ID,9.0,16.0,26.0,39.0,42.0,51.0,97.0,99.0,114.0,125.0,...,278773.0,278798.0,278800.0,278807.0,278828.0,278832.0,278836.0,278843.0,278844.0,278854.0
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [207]:
countries_combine_pivot.shape

(673, 20873)

In [208]:
from sklearn.neighbors import NearestNeighbors

knn_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
knn_model.fit(countries_combine_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [209]:
query_index = np.random.choice(countries_combine_pivot.shape[0])
print(query_index)
print(countries_combine_pivot.index[query_index])
countries_combine_pivot.iloc[query_index:].values.reshape(1,-1)

628
To Kill a Mockingbird


array([[ 0.,  0., 10., ...,  0.,  0.,  0.]])

In [210]:
distances, indices = knn_model.kneighbors(countries_combine_pivot.iloc[query_index,:].values.reshape(1,-1), n_neighbors= 11)


In [211]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(countries_combine_pivot.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, countries_combine_pivot.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for To Kill a Mockingbird:

1: Lord of the Flies, with distance of 0.8983332784791878:
2: The Catcher in the Rye, with distance of 0.9164373015548204:
3: Animal Farm, with distance of 0.9181880394282356:
4: Fahrenheit 451, with distance of 0.9219054060860632:
5: Tears of the Giraffe (No.1 Ladies Detective Agency), with distance of 0.9305766147901575:
6: Of Mice and Men (Penguin Great Books of the 20th Century), with distance of 0.9312892746220178:
7: STONES FROM THE RIVER, with distance of 0.9349949202758574:
8: The Remains of the Day (Vintage International), with distance of 0.9350298018095347:
9: The Firm, with distance of 0.9360677546368776:
10: Five Quarters of the Orange, with distance of 0.9371958400628537:


In [212]:
#book_names_list=[]
#for i in range(1, len(distances.flatten())):
    #book_names_list.append(countries_combine_pivot.index[indices.flatten()[i]])
#book_names_list

In [213]:
book_names_list = list(countries_combine_pivot.index[indices.flatten()])
book_names_list

['To Kill a Mockingbird',
 'Lord of the Flies',
 'The Catcher in the Rye',
 'Animal Farm',
 'Fahrenheit 451',
 'Tears of the Giraffe (No.1 Ladies Detective Agency)',
 'Of Mice and Men (Penguin Great Books of the 20th Century)',
 'STONES FROM THE RIVER',
 'The Remains of the Day (Vintage International)',
 'The Firm',
 'Five Quarters of the Orange']

In [214]:
final  = books1[books1['Book-Title'].isin(book_names_list)].groupby('Book-Title').first()
final.reset_index(inplace=True)
final

,Book-Title,ISBN,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,Animal Farm,0451526341,George Orwell,2004,Signet,http://images.amazon.com/images/P/0451526341.0...
1,Fahrenheit 451,3257208626,Ray Bradbury,1994,Distribooks Inc,http://images.amazon.com/images/P/3257208626.0...
2,Five Quarters of the Orange,0060958022,Joanne Harris,2002,Perennial,http://images.amazon.com/images/P/0060958022.0...
3,Lord of the Flies,0399501487,William Gerald Golding,1959,Perigee Trade,http://images.amazon.com/images/P/0399501487.0...
4,Of Mice and Men (Penguin Great Books of the 20...,0140177396,John Steinbeck,1993,Penguin Books,http://images.amazon.com/images/P/0140177396.0...
5,STONES FROM THE RIVER,0684844729,Ursula Hegi,1997,Simon &amp; Schuster,http://images.amazon.com/images/P/0684844729.0...
6,Tears of the Giraffe (No.1 Ladies Detective Ag...,1400031354,Alexander McCall Smith,2002,Anchor,http://images.amazon.com/images/P/1400031354.0...
7,The Catcher in the Rye,0316769487,J.D. Salinger,1991,"Little, Brown",http://images.amazon.com/images/P/0316769487.0...
8,The Firm,0385416342,John Grisham,1991,Doubleday Books,http://images.amazon.com/images/P/0385416342.0...
9,The Remains of the Day (Vintage International),0679731725,Kazuo Ishiguro,1993,Vintage Books USA,http://images.amazon.com/images/P/0679731725.0...


In [215]:
# combine_books

In [216]:
combine_books11 = combine_books[combine_books['Book-Rating']>=1]

In [217]:
title_list = list(set(combine_books11['Book-Title'].tolist()))

In [218]:
title_list[:5]

['Postmortem',
 'Love in the Time of Cholera (Penguin Great Books of the 20th Century)',
 'Bee Season: A Novel',
 'The Burden of Proof',
 'Desperation']

In [219]:
combine_books11[combine_books11['Book-Title']==title_list[9]]

,User-ID,ISBN,Book-Rating,Book-Title
113652,11676,0446608815,6,Pop Goes the Weasel
113653,12641,0446608815,7,Pop Goes the Weasel
113656,26280,0446608815,8,Pop Goes the Weasel
113657,28177,0446608815,6,Pop Goes the Weasel
113661,33124,0446608815,8,Pop Goes the Weasel
...,...,...,...,...
113817,266056,0446608815,9,Pop Goes the Weasel
113818,269410,0446608815,10,Pop Goes the Weasel
113820,269890,0446608815,8,Pop Goes the Weasel
113821,270906,0446608815,7,Pop Goes the Weasel


In [220]:
avg_rating = []

for i in title_list:
    avg_rating.append([i,round(combine_books11[combine_books11['Book-Title']==i]['Book-Rating'].mean(),1)])

In [221]:
avg_rating[:10]

[['Postmortem', 7.9],
 ['Love in the Time of Cholera (Penguin Great Books of the 20th Century)',
  8.1],
 ['Bee Season: A Novel', 7.5],
 ['The Burden of Proof', 7.1],
 ['Desperation', 7.3],
 ['Three Junes', 7.3],
 ['The Bluest Eye', 7.4],
 ['Running with Scissors: A Memoir', 7.5],
 ['Along Came a Spider (Alex Cross Novels)', 8.0],
 ['Pop Goes the Weasel', 7.4]]

In [222]:
books2 = books1.dropna()

In [223]:
# books1[books1['Book-Title']==avg_rating[0][0]]['Avg-Rating'] = avg_rating[0][1]

for i in avg_rating:
    index_list = list(books2[books2['Book-Title']==i[0]].index)
    books2.loc[index_list, 'Avg-Rating'] = i[1]

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [224]:
books2.iloc[8690]

ISBN                                                          0446610542
Book-Title              The Bachelor (Warner Books Contemporary Romance)
Book-Author                                               Carly Phillips
Year-Of-Publication                                                 2002
Publisher                                                 Warner Forever
Image-URL-L            http://images.amazon.com/images/P/0446610542.0...
Avg-Rating                                                           7.3
Name: 8690, dtype: object

In [225]:
print(books2['Avg-Rating'].isna().sum())

books2.info()

269439
<class 'pandas.core.frame.DataFrame'>
Int64Index: 271354 entries, 0 to 271359
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 271354 non-null  object 
 1   Book-Title           271354 non-null  object 
 2   Book-Author          271354 non-null  object 
 3   Year-Of-Publication  271354 non-null  object 
 4   Publisher            271354 non-null  object 
 5   Image-URL-L          271354 non-null  object 
 6   Avg-Rating           1915 non-null    float64
dtypes: float64(1), object(6)
memory usage: 24.6+ MB


In [226]:
type(avg_rating_dict[555])

numpy.float64

In [227]:
['Avg-Rating'] = a

SyntaxError: cannot assign to literal (Temp/ipykernel_16860/2679819279.py, line 1)

In [ ]:
books1[books1['Book-Title']=='Postmortem']

In [ ]:
books1

In [ ]:
[[a,3][b,6]]

In [228]:
combine_books1.where(combine_books1["Book-Rating"]>=1, inplace = True)
operation1 = combine_books1.dropna().sort_values(by = 'Book-Rating',ascending=False).drop(columns = 'Total_Rating_Count')
operation1

,User-ID,ISBN,Book-Rating,Book-Title
4,7158.0,0446520802,10.0,The Notebook
58572,177432.0,0449212602,10.0,The Handmaid's Tale
58622,235004.0,0449212602,10.0,The Handmaid's Tale
58604,218568.0,0449212602,10.0,The Handmaid's Tale
58589,204864.0,0449212602,10.0,The Handmaid's Tale
...,...,...,...,...
52919,179019.0,0385511612,1.0,Bleachers
9296,142600.0,0971880107,1.0,Wild Animus
52955,251843.0,0385511612,1.0,Bleachers
52968,274656.0,0385511612,1.0,Bleachers


In [229]:
operation2 = operation1[operation1['Book-Title']=='The Notebook']

avg_rating_count = round(operation2["Book-Rating"].mean(),3)
operation2['Avg_Rating']= avg_rating_count

C:\Users\Lenovo\AppData\Local\Temp/ipykernel_16860/2369478290.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  operation2['Avg_Rating']= avg_rating_count


In [230]:
operation2

,User-ID,ISBN,Book-Rating,Book-Title,Avg_Rating
4,7158.0,0446520802,10.0,The Notebook,7.844
5,8253.0,0446520802,10.0,The Notebook,7.844
175,36606.0,0446605239,10.0,The Notebook,7.844
462,212260.0,0446605239,10.0,The Notebook,7.844
243,74758.0,0446605239,10.0,The Notebook,7.844
...,...,...,...,...,...
525,249246.0,0446605239,3.0,The Notebook,7.844
100,245568.0,0446520802,2.0,The Notebook,7.844
262,93631.0,0446605239,2.0,The Notebook,7.844
187,41324.0,0446605239,2.0,The Notebook,7.844


In [231]:
operation3 = operation1['Book-Title'].to_list()

In [232]:
for i in operation3[:10]:
    operation2 = operation1[operation1['Book-Title']==i]
print(operation2)
#     avg_rating_count = round(operation2["Book-Rating"].mean(),3)
#     print(avg_rating_count)
#     operation2['Avg_Rating']= avg_rating_count

        User-ID        ISBN  Book-Rating           Book-Title
58572  177432.0  0449212602         10.0  The Handmaid's Tale
58622  235004.0  0449212602         10.0  The Handmaid's Tale
58604  218568.0  0449212602         10.0  The Handmaid's Tale
58589  204864.0  0449212602         10.0  The Handmaid's Tale
58584  197364.0  0449212602         10.0  The Handmaid's Tale
...         ...         ...          ...                  ...
58588  201637.0  0449212602          6.0  The Handmaid's Tale
58457   60211.0  0449212602          5.0  The Handmaid's Tale
58491   96608.0  0449212602          5.0  The Handmaid's Tale
58487   93629.0  0449212602          4.0  The Handmaid's Tale
58657  266299.0  0449212602          4.0  The Handmaid's Tale

[111 rows x 4 columns]


In [233]:
operation2

,User-ID,ISBN,Book-Rating,Book-Title
58572,177432.0,0449212602,10.0,The Handmaid's Tale
58622,235004.0,0449212602,10.0,The Handmaid's Tale
58604,218568.0,0449212602,10.0,The Handmaid's Tale
58589,204864.0,0449212602,10.0,The Handmaid's Tale
58584,197364.0,0449212602,10.0,The Handmaid's Tale
...,...,...,...,...
58588,201637.0,0449212602,6.0,The Handmaid's Tale
58457,60211.0,0449212602,5.0,The Handmaid's Tale
58491,96608.0,0449212602,5.0,The Handmaid's Tale
58487,93629.0,0449212602,4.0,The Handmaid's Tale


In [241]:
countries_combine.head(5)

,User-ID,ISBN,Book-Rating,Book-Title,Total_Rating_Count,country
0,7158.0,0446520802,10.0,The Notebook,581.0,usa
2,7158.0,0446605484,8.0,Roses Are Red (Alex Cross Novels),230.0,usa
3,7158.0,0316693235,5.0,Violets Are Blue,370.0,usa
4,7158.0,074343627X,1.0,Dreamcatcher,452.0,usa
5,7158.0,0671510053,9.0,SHIPPING NEWS,349.0,usa


In [242]:
countries_combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52368 entries, 0 to 57169
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   User-ID             52368 non-null  float64
 1   ISBN                52368 non-null  object 
 2   Book-Rating         52368 non-null  float64
 3   Book-Title          52368 non-null  object 
 4   Total_Rating_Count  52368 non-null  float64
 5   country             52368 non-null  object 
dtypes: float64(3), object(3)
memory usage: 2.8+ MB


In [244]:
countries_combine_ISBN_list = list(set(countries_combine.ISBN))

In [246]:
books2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271354 entries, 0 to 271359
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   ISBN                 271354 non-null  object 
 1   Book-Title           271354 non-null  object 
 2   Book-Author          271354 non-null  object 
 3   Year-Of-Publication  271354 non-null  object 
 4   Publisher            271354 non-null  object 
 5   Image-URL-L          271354 non-null  object 
 6   Avg-Rating           1915 non-null    float64
dtypes: float64(1), object(6)
memory usage: 24.6+ MB


In [238]:
len(set(books2.ISBN))

271354

In [247]:
books2 = books2[books2['ISBN'].isin(countries_combine_ISBN_list)]

In [248]:
books2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 727 entries, 18 to 35704
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ISBN                 727 non-null    object 
 1   Book-Title           727 non-null    object 
 2   Book-Author          727 non-null    object 
 3   Year-Of-Publication  727 non-null    object 
 4   Publisher            727 non-null    object 
 5   Image-URL-L          727 non-null    object 
 6   Avg-Rating           727 non-null    float64
dtypes: float64(1), object(6)
memory usage: 45.4+ KB


In [251]:
books2['Avg-Rating']

18       7.6
19       7.8
26       4.4
27       7.5
28       7.9
        ... 
28071    7.3
29214    7.8
30534    8.2
30774    7.3
35704    7.8
Name: Avg-Rating, Length: 727, dtype: float64

In [252]:
books2

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Avg-Rating
18,0440234743,The Testament,John Grisham,1999,Dell,http://images.amazon.com/images/P/0440234743.0...,7.6
19,0452264464,Beloved (Plume Contemporary Fiction),Toni Morrison,1994,Plume,http://images.amazon.com/images/P/0452264464.0...,7.8
26,0971880107,Wild Animus,Rich Shapero,2004,Too Far,http://images.amazon.com/images/P/0971880107.0...,4.4
27,0345402871,Airframe,Michael Crichton,1997,Ballantine Books,http://images.amazon.com/images/P/0345402871.0...,7.5
28,0345417623,Timeline,MICHAEL CRICHTON,2000,Ballantine Books,http://images.amazon.com/images/P/0345417623.0...,7.9
...,...,...,...,...,...,...,...
28071,0425178765,Easy Prey,John Sandford,2001,Berkley Publishing Group,http://images.amazon.com/images/P/0425178765.0...,7.3
29214,0449223604,M Is for Malice,Sue Grafton,1998,Fawcett Books,http://images.amazon.com/images/P/0449223604.0...,7.8
30534,0345444884,The Talisman,STEPHEN KING,2001,Ballantine Books,http://images.amazon.com/images/P/0345444884.0...,8.2
30774,0060008032,Angels,Marian Keyes,2003,HarperTorch,http://images.amazon.com/images/P/0060008032.0...,7.3


In [253]:
final = books2[books2['Book-Title'].isin(book_names_list)].groupby('Book-Title').first()

In [257]:
def getImageURL(input1):
    return books2[books2['ISBN']==input1]['Image-URL-L']

In [ ]:
getImageURL()